# A Brute Approach & Optimizer
## Formerly, Swap and Give Greedy Optimizer

In [ ]:
from haversine import haversine
import seaborn as sns
import pandas as pd
import numpy as np
import sqlite3
import math
import time

%matplotlib inline
import matplotlib.pyplot as plt

# The optimization functions
##  - The Magic happens here

In [ ]:

north_pole = (90,0)
weight_limit = 1000.0

def bb_sort(ll):
    s_limit = 100
    optimal = False
    ll = [[0,north_pole,10]] + ll[:] + [[0,north_pole,10]] 
    while not optimal:
        optimal = True
        for i in range(1,len(ll) - 2):
            lcopy = ll[:]
            lcopy[i], lcopy[i+1] = ll[i+1][:], ll[i][:]
            if path_opt_test(ll[1:-1])[0] > path_opt_test(lcopy[1:-1])[0]:
                #print("Sort Swap")
                ll = lcopy[:]
                optimal = False
                s_limit -= 1
                if s_limit < 0:
                    optimal = True
                    break
    return ll[1:-1]

def prev_path_opt(curr,prev):
    curr = [[0,north_pole,10]] + curr[:] + [[0,north_pole,10]]
    prev = [[0,north_pole,10]] + prev[:] + [[0,north_pole,10]]
    for curr_ in range(1,len(curr) - 2):
        for prev_ in range(1,len(prev) - 2):
            lcopy_curr = curr[:]
            lcopy_prev = prev[:]
            lcopy_curr[curr_], lcopy_prev[prev_] = lcopy_prev[prev_][:], lcopy_curr[curr_][:]
            if ((path_opt_test(lcopy_curr[1:-1])[0] + path_opt_test(lcopy_prev[1:-1])[0]) < (path_opt_test(curr[1:-1])[0] + path_opt_test(prev[1:-1])[0])) and path_opt_test(lcopy_curr[1:-1])[2] <=1000 and path_opt_test(lcopy_prev[1:-1])[2] <= 1000:
                #print("Trip Swap")
                curr = lcopy_curr[:]
                prev = lcopy_prev[:]
    return [curr[1:-1], prev[1:-1]]

def prev_path_opt_s1(curr,prev):
    curr = [[0,north_pole,10]] + curr[:] + [[0,north_pole,10]]
    prev = [[0,north_pole,10]] + prev[:] + [[0,north_pole,10]]
    for curr_ in range(1,len(curr) - 1):
        for prev_ in range(1,len(prev) - 1):
            lcopy_curr = curr[:]
            lcopy_prev = prev[:]
            if len(lcopy_prev)-1 <= prev_:
                break
            lcopy_curr = lcopy_curr[:curr_+1][:] + [lcopy_prev[prev_]] + lcopy_curr[curr_+1:][:]
            lcopy_prev.pop(prev_)
            if ((path_opt_test(lcopy_curr[1:-1])[0] + path_opt_test(lcopy_prev[1:-1])[0]) <= (path_opt_test(curr[1:-1])[0] + path_opt_test(prev[1:-1])[0])) and path_opt_test(lcopy_curr[1:-1])[2] <=1000 and path_opt_test(lcopy_prev[1:-1])[2] <= 1000:
                print("Trip Swap - Give to current")
                curr = lcopy_curr[:]
                prev = lcopy_prev[:]
    return [curr[1:-1], prev[1:-1]]

def split_trips(curr):
    prev = []
    curr = [[0,north_pole,10]] + curr[:] + [[0,north_pole,10]]
    for curr_ in range(1,len(curr) - 1):
        lcopy_curr = curr[:]
        if len(lcopy_curr)-1 <=curr_:
            break
        lcopy_prev = [[0,north_pole,10]] + [lcopy_curr[curr_]] + [[0,north_pole,10]]
        lcopy_curr.pop(curr_)
        if ((path_opt_test(lcopy_curr[1:-1])[0] + path_opt_test(lcopy_prev[1:-1])[0]) < (path_opt_test(curr[1:-1])[0])):
            print("Trip Split")
            curr = lcopy_curr[:]
            prev = lcopy_prev[:]
    return [curr[1:-1], prev[1:-1]]

def path_opt_test(llo):
    f_ = 0.0
    d_ = 0.0
    we_ = 0.0
    l_ = north_pole
    for i in range(len(llo)):
        d_ += haversine(l_, llo[i][1])
        we_ += llo[i][2]
        f_ += d_ * llo[i][2]
        l_ = llo[i][1]
    d_ += haversine(l_, north_pole)
    f_ += d_ * 10
    return [f_,d_,we_]

# Slicing the Sphere
## Almost everything can be scaled in a grid or a Matrix